In [1]:
# ! wget http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip

In [2]:
# ! ls

In [3]:
# ! apt install unzip
# ! unzip cornell_movie_dialogs_corpus.zip

In [4]:
# ! ls

In [5]:
# ! wget http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/mono/OpenSubtitles.raw.en.gz

In [6]:
# ! ls

In [7]:
# !tar -xvf  'download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en.gz' -C 'data'

In [8]:
! wget https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/en.txt.gz

--2020-06-19 18:27:19--  https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/en.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3663376519 (3.4G) [application/gzip]
Saving to: ‘en.txt.gz’

en.txt.gz           100%[===================>]   3.41G  22.0MB/s    in 2m 39s  

2020-06-19 18:29:58 (22.0 MB/s) - ‘en.txt.gz’ saved [3663376519/3663376519]



In [9]:
! apt install gunzip
! gunzip en.txt.gz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package gunzip


In [11]:
# % cd ..

In [12]:
# ! wget https://github.com/PolyAI-LDN/conversational-datasets/blob/master/opensubtitles/create_data.py

In [43]:
import os
import re
from os import path
import tensorflow as tf
import numpy as np

In [44]:
def _should_skip(line, min_length, max_length):
    """Whether a line should be skipped depending on the length."""
    return len(line) < min_length or len(line) > max_length

In [45]:
def create_example(previous_lines, line, file_id):
    """Creates examples with multi-line context
    The examples will include:
        file_id: the name of the file where these lines were obtained.
        response: the current line text
        context: the previous line text
        context/0: 2 lines before
        context/1: 3 lines before, etc.
    """

    example = {
        'file_id': file_id,
        'context': previous_lines[-1],
        'response': line,
    }
    example['file_id'] = file_id
    #print(file_id)
    example['context'] = previous_lines[-1]
    #print(previous_lines[-1])
    extra_contexts = previous_lines[:-1]
    example.update({
        'context/{}'.format(i): context
        for i, context in enumerate(extra_contexts[::-1])
    })

    return example

In [46]:
def _preprocess_line(line):
    #line = line.decode("utf-8")

    # Remove the first word if it is followed by colon (speaker names)
    # NOTE: this wont work if the speaker's name has more than one word
    line = re.sub('(?:^|(?:[.!?]\\s))(\\w+):', "", line)

    # Remove anything between brackets (corresponds to acoustic events).
    line = re.sub("[\\[(](.*?)[\\])]", "", line)

    # Strip blanks hyphens and line breaks
    line = line.strip(" -\n")

    return line

In [48]:
def _create_examples_from_file(file_name,file_id, min_length=0, max_length=24,
                               num_extra_contexts=2):
   # _, file_id = path.split(file_name)
    #print(file_id,"#")
    previous_lines = []
    for line in open(file_name):
        line = _preprocess_line(line)
        
        if not line:
            continue

        should_skip = _should_skip(
            line,
            min_length=min_length,
            max_length=max_length)

        if previous_lines:
            should_skip |= _should_skip(
                previous_lines[-1],
                min_length=min_length,
                max_length=max_length)

            if not should_skip:
                yield create_example(previous_lines, line, file_id)

        previous_lines.append(line)
        if len(previous_lines) > num_extra_contexts + 1:
            del previous_lines[0]

In [49]:
def _features_to_serialized_tf_example(features):
    """Convert a string dict to a serialized TF example.
    The dictionary maps feature names (strings) to feature values (strings).
    """
    #print("hello")
    example = tf.train.Example()
    for feature_name, feature_value in features.items():
        example.features.feature[feature_name].bytes_list.value.append(
            feature_value.encode("utf-8"))
    return example.SerializeToString()


def _shuffle_examples(examples):
    examples |= ("add random key" >> beam.Map(
        lambda example: (uuid.uuid4(), example)))
    examples |= ("group by key" >> beam.GroupByKey())
    examples |= ("get shuffled values" >> beam.FlatMap(lambda t: t[1]))
    return examples

In [50]:
# % cd ..

In [51]:
# ! ls

In [52]:
# ! pwd

In [53]:
# ! ( head -1000000 en.txt ; ) > million.txt

In [62]:
test=_create_examples_from_file(file_name='en.txt',file_id='en.txt', num_extra_contexts=0)

In [88]:
alpha=[]
sent=[]
count=0
num_examples = 10**1
for x in test:
  count+=1
  if count<num_examples:
    alpha.append(x)  
    sent.append  
  else:
    break

In [89]:
len(alpha)

9

In [91]:
X_train[0]

"We're not?"

In [92]:
dat=open('en.txt')
sents=[]
for line in dat:
  sents.append(_preprocess_line(line))
  if len(sents)>num_examples-1:
    break

In [93]:
len(sents)

10

In [219]:
import itertools
import collections
import numpy as np
import nltk
nltk.download('punkt')
SENT_START_TOKEN = "SENTENCE_START"
SENT_END_TOKEN = "SENTENCE_END"
UNKNOWN_TOKEN = "UNKNOWN_TOKEN"
PADDING_TOKEN = "PADDING"


def tokenize_text(text_lines):
    """
    Split text into sentences, append start and end tokens to each and tokenize
    :param text_lines: list of text lines or list of length one containing all text
    :return: list of sentences
    """
    sentences = itertools.chain(*[nltk.sent_tokenize(line.lower()) for line in text_lines])
    sentences = ["{} {} {}".format(SENT_START_TOKEN, x, SENT_END_TOKEN) for x in sentences]
    tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]
    for i in range(len(tokenized_sentences)):
      while(len(tokenized_sentences[i])<24):
        tokenized_sentences[i].append(PADDING_TOKEN)
    return tokenized_sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [220]:
t_sents=tokenize_text(sents)

In [221]:
print(t_sents[0])

['SENTENCE_START', 'presented', 'by', 'im', 'pictures', 'SENTENCE_END', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING']


In [222]:
def get_words_mappings(tokenized_sentences, vocabulary_size):
    # Using NLTK
    # frequence = nltk.FreqDist(itertools.chain(*tokenized_sentences))
    # vocab = frequence.most_common(vocabulary_size)

    # Using basic counter
    counter = collections.Counter(itertools.chain(*tokenized_sentences))
    vocab = counter.most_common(vocabulary_size)
    index_to_word = [x[0] for x in vocab]
    # Add padding for index 0
    index_to_word.insert(0, PADDING_TOKEN)
    # Append unknown token (with index = vocabulary size + 1)
    index_to_word.append(UNKNOWN_TOKEN)
    word_to_index = dict([(w, i) for i, w in enumerate(index_to_word)])
    return index_to_word, word_to_index

In [223]:
vocabulary_size=10**4

In [224]:
id2w,w2id=get_words_mappings(t_sents,vocabulary_size=vocabulary_size)

In [225]:
id2w[0]

'PADDING'

In [226]:
tok_sents=[]
for sent in t_sents:
  sent=[w2id[w] for w in sent]
  tok_sents.append(sent)

In [227]:
print(len(tok_sents))

10


In [228]:
X_train = np.array([tok_sents[i] for i in range(0, len(tok_sents)-1)])
Y_train = np.array([tok_sents[i] for i in range(1,len(tok_sents))])

In [229]:
print(len(X_train),len(Y_train))
print(X_train[0],Y_train[0])

9 9
[ 2 16  7 17 18  3  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1] [ 2 19  7 20 21  3  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1]


In [230]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2020-06-19 20:15:30--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-06-19 20:15:31--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-06-19 20:15:31--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove

In [231]:
glove_dir = './'

embeddings_index = {} #initialize dictionary
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [233]:
embedding_dim = 100
num_words= vocabulary_size
embedding_matrix = np.zeros((num_words, embedding_dim)) #create an array of zeros with word_num rows and embedding_dim columns
for i, word in enumerate(id2w):
    embedding_vector = embeddings_index.get(word)
    if i < num_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [241]:
Y_train = np.expand_dims(Y_train, -1)

In [234]:
import keras.backend as K
from keras.preprocessing import sequence
from keras.models import model_from_json
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, BatchNormalization
from keras.layers.core import Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.layers import LSTM, TimeDistributed, RepeatVector, Input

In [235]:
toyModel = models.Sequential()
toyModel.add(Embedding(num_words, 
                    embedding_dim, 
                    input_length=24,
                      weights=[embedding_matrix],
                      trainable=False))

In [236]:
toyModel.add(BatchNormalization())
#model.add(TimeDistributed(Flatten())) # not needed if proper batch_input_shape specified before
toyModel.add(LSTM(256, 
               return_sequences=True, 
               stateful=False, # if stateful model, remember to avoid batches shuffling during training
              ))#activation='relu')) ??easily getting loss=nan if using RELU
toyModel.add(TimeDistributed(Dense(vocabulary_size, activation='softmax')))

In [237]:
toyModel.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam')

In [238]:
toyModel.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 24, 100)           1000000   
_________________________________________________________________
batch_normalization_5 (Batch (None, 24, 100)           400       
_________________________________________________________________
lstm_3 (LSTM)                (None, 24, 256)           365568    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 24, 10000)         2570000   
Total params: 3,935,968
Trainable params: 2,935,768
Non-trainable params: 1,000,200
_________________________________________________________________


In [244]:
num_epoch = 100
toyModel.fit(X_train, 
          Y_train, 
          epochs=num_epoch, 
          batch_size=1, 
         )

Epoch 1/100
9/9 [==============================] - 1s 68ms/step - loss: 1.7354
Epoch 2/100
9/9 [==============================] - 1s 72ms/step - loss: 1.6141
Epoch 3/100
9/9 [==============================] - 1s 68ms/step - loss: 1.5632
Epoch 4/100
9/9 [==============================] - 1s 70ms/step - loss: 1.4763
Epoch 5/100
9/9 [==============================] - 1s 72ms/step - loss: 1.3780
Epoch 6/100
9/9 [==============================] - 1s 75ms/step - loss: 1.2757
Epoch 7/100
9/9 [==============================] - 1s 76ms/step - loss: 1.2081
Epoch 8/100
9/9 [==============================] - 1s 72ms/step - loss: 1.1447
Epoch 9/100
9/9 [==============================] - 1s 71ms/step - loss: 1.0479
Epoch 10/100
9/9 [==============================] - 1s 74ms/step - loss: 0.9829
Epoch 11/100
9/9 [==============================] - 1s 71ms/step - loss: 0.9124
Epoch 12/100
9/9 [==============================] - 1s 72ms/step - loss: 0.8627
Epoch 13/100
9/9 [==============================]

In [245]:
import numpy as np
import re
import nltk

import logging
logging.basicConfig(level=logging.DEBUG)


class TextGenModel:
    SENT_START_TOKEN = "SENTENCE_START"
    SENT_END_TOKEN = "SENTENCE_END"
    UNKNOWN_TOKEN = "UNKNOWN_TOKEN"
    PAD_TOKEN = "PADDING"

    TYPES = {'delimited': 0,  # characterized by start and end sentence tokens
             'continuous': 1,  # no delimiter tokens
             }

    def __init__(self, model, index_to_word, word_to_index, sent_max_len=30, temperature=1.0,
                 use_embeddings=False, model_type=TYPES['delimited']):
        self.model = model
        self.index_to_word = index_to_word
        self.word_to_index = word_to_index
        self.vocabulary_size = len(word_to_index)
        self.unknown_token_idx = self.word_to_index[self.UNKNOWN_TOKEN]
        self.pad_token_idx = self.word_to_index[self.PAD_TOKEN]

        self.start_token_idx = self.word_to_index.get(self.SENT_START_TOKEN, None)
        self.end_token_idx = self.word_to_index.get(self.SENT_END_TOKEN, None)

        self.sent_max_len = sent_max_len
        self.temperature = temperature
        self.use_embeddings = use_embeddings
        self.model_type = model_type

    def get_sentence(self, sent_min_len, seed=None, seed_max_len = 10):
        """
        Returns a sentence generated by the model.
        Method will loop on generated sentences until requirements are met.
        :param sent_min_len: minimum length acceptable for the sentence
        :param seed: seed text to use for the generation task
        :param seed_max_len: max length used for the seed (operated pre-truncation)
        :return: the generated string
        """
        if seed == '':
            seed = None

        start_sentence = self._generate_start_sentence(seed_max_len, seed)

        sent = None
        while not sent:
            sent = self._generate_sentence(sent_min_len, start_sentence)
        return sent

    def _generate_start_sentence(self, max_len, seed=None):
        """
        Generate sentence based on seed text.
        Sentence should then be feed to the model for the actual text generation task.
        """
        # if we have some seed text, start with that
        if seed:
            idx_sentence = self.transform_sentence(seed, max_len)
        else:
            # if present use start token
            if self.start_token_idx:
                start_token_idx = self.start_token_idx
            # otherwise use random token index
            else:
                start_token_idx = np.random.randint(self.vocabulary_size)
            idx_sentence = np.array([start_token_idx])
        # one hot encode sentence
        if not self.use_embeddings:
            return self.one_hot_encode_sentence(idx_sentence)
        else:
            return idx_sentence

    # TODO improve on prediction, UNKNOWN handling and timeout
    def _generate_sentence(self, min_len, seed_sentence):
        """
        Main procedure for sentence generation.
        :param min_len: minimum length acceptable for the generated sentence
        :param seed_sentence: seed sentence on which to build newly generated text (can be either one-hot or word-index encoded)
        :return:
        """
        # seed_sentence holds all history of words,
        # res_sentence is instead the final resulting response sentence
        res_sentence = []

        # Repeat until stopping criteria are met
        while True:
            # Index from which we will sample is the one corresponding to index of last
            # word in the fed sentence (RNN next predicted word)
            current_idx = max(0, len(seed_sentence) - 1)

            # Make predictions and sample index based on returned probabilities
            words_probs = self._predict_fun(seed_sentence, idx=current_idx)
            sampled_index = TextGenModel.sample_from_prediction(words_probs, temperature=self.temperature)

            # TODO could try some more sampling before throwing away already done work?
            #Skip if sentence is getting too long or we got an unwanted token
            if len(res_sentence) >= self.sent_max_len \
                    or sampled_index == self.unknown_token_idx \
                    or sampled_index == self.pad_token_idx:
                return None

            # Append result to sentences
            res_sentence.append(sampled_index)
            # append also to seed sentence (just index if with embedding, one-hot encoded if without
            if self.use_embeddings:
                seed_sentence = np.append(seed_sentence, [sampled_index], axis=0)
            else:
                seed_sentence = np.append(seed_sentence, [self.one_hot_encode_word(sampled_index)], axis=0)

            # Return if we get an end token and sentence is long enough
            if len(res_sentence) > min_len and (not self.end_token_idx or
                                                        sampled_index == self.end_token_idx):
                return res_sentence

    def _generate_answer(self, min_len, oh_question):
        # Make predictions and sample index based on probs
        answer_probs = self._predict_fun(oh_question)[0]

        # Repeat until we get an end token
        while True:
            idx_sentence = []
            for words_probs in answer_probs:
                sampled_index = self.sample_from_prediction(words_probs, temperature=self.temperature)

                idx_sentence.append(sampled_index)

                # Skip if sentence is getting too long or we got an unknown token
                if sampled_index == self.unknown_token_idx:
                    break
                # Return if we get an end token and sentence is long enough
                if sampled_index == self.end_token_idx and len(idx_sentence) > min_len:
                    return idx_sentence

    # TODO add padding for models who need fixed size input
    def _predict_fun(self, sentence, idx=None):
        """
        Model prediction based on give sentence
        :param sentence: sentence to use (one-hot encoded)
        :param idx: if specified, return only probabilities for such index
        """
        # Give required shape (sample=1, sent_len, voc_size)
        x = np.expand_dims(sentence, 0)
        predictions = self.model.predict(x)
        if not idx is None:
            return predictions[0][idx]
        else:
            return predictions

    @staticmethod
    def sample_from_prediction(predictions, temperature=1.0):
        """
        Sample an index from given prediction (a probability distribution)
        :param predictions:
        :param temperature:
        :return:
        """
        p = np.log(np.asarray(predictions).astype('float64')) / temperature
        p = np.exp(p) / np.sum(np.exp(p))
        try:
            idx = np.argmax(np.random.multinomial(1, p, 1))
        except ValueError as e:
            logging.debug(e)
            return np.random.choice(len(predictions), 1, p=predictions)[0]
        return idx

    def transform_sentence(self, sentence, max_len):
        """
        Takes a sentence as string and transform it as required by the model
        (tokenize, words to index)
        """
        # Tokenize
        # TODO use only spacy
        words = nltk.word_tokenize(sentence)  # consider adding lower
        # Words to index
        idx_sentence = [self.word_to_index.get(w, self.unknown_token_idx) for w in words[-max_len:]]
        return idx_sentence

    def one_hot_encode_sentence(self, sentence):
        """
        Encode each word of the sentence to one-hot representation.
        :param sentence: sentence in words indexes representation
        """
        return np.eye(self.vocabulary_size)[sentence]

    def one_hot_encode_word(self, word_idx):
        """
        Encode word to one-hot representation.
        :param word_idx: word index to encode
        """
        oh_word = np.zeros(self.vocabulary_size)
        if word_idx >= self.vocabulary_size:
            raise Exception("Word index {} is out of range given a vocabulary size of {}"
                            .format(word_idx, self.vocabulary_size))
        else:
            oh_word[word_idx] = 1.0
        return oh_word

    def _sample_word_idx_from(self, words_probs):
        unknown_token_idx = self.word_to_index[self.UNKNOWN_TOKEN]
        sampled_index = unknown_token_idx
        # Sample until a known word is found
        while sampled_index == unknown_token_idx:
            samples = np.random.multinomial(1, words_probs[-1])
            sampled_index = np.argmax(samples)
        return sampled_index

    # prettify raw generated sentence to string
    def pretty_print_sentence(self, sentence, text_max_len=None):
        # if model of type delimited, remove end token
        if self.model_type == self.TYPES['delimited']:
            sentence = sentence[:-1]
        # if specified, remove unwanted tail text
        if text_max_len:
            sentence = sentence[:text_max_len]
        # convert sentence from word_indexes to string, and tries to fix spacing
        words = [self.index_to_word[word_idx].strip() for word_idx in sentence]
        words = [w if re.match(r"[\,!\?\':\.]+|n'", w) else ' ' + w for w in
                 words]
        # return joined words (spaced added before)
        return "".join(words).strip()

In [247]:
trained_epochs=100

In [252]:
pred=toyModel.predict(X_train)

In [254]:
pred = np.argmax(pred, axis = -1)
output = [id2w[int(i)] for i in pred]

In [255]:
print(output)

['.', 'two', '.', 'SENTENCE_END', 'i', "n't", ',', 'SENTENCE_END', ',']
